# Association Rule Mining

## Overview

Associations are relationships between objects. The idea behind association rule mining is to determine rules, that allow us to identify which objects may be related to a set of objects were already know. In the association mining terminology, we refer to the objects as *items*. A common example for association rule mining is basket analysis. A shopper puts items from a store into a basket. Once there are some items in the basket, it is possible to recommend associated items that are available in the store to the shopper. 

<img src="images/associationsrules_general.png" alt="Example for Associations" style="width: 500px;"/>

In this example, the association between items is defined as "shoppers buy items together". More generally speaking, we have *transactions*, and in each transaction we observe a set of related objects. We apply association rule mining to a set of transactions to infer *association rules* that describe the associations between items. 

<img src="images/associationsrules_abstract.png" alt="Association Rule Mining" style="width: 500px;"/>